In [ ]:
# ============================================================
# 🔥 FINAL WORKING CNN MODEL FOR PREDICTIVE MAINTENANCE
# ============================================================

import warnings
warnings.filterwarnings('ignore')

# -------------------- 1. Upload TRAIN & TEST CSV --------------------
from google.colab import files
import pandas as pd
import numpy as np

print("📌 Upload TRAIN dataset CSV")
uploaded_train = files.upload()
train_path = list(uploaded_train.keys())[0]

print("\n📌 Upload TEST dataset CSV")
uploaded_test = files.upload()
test_path = list(uploaded_test.keys())[0]

# -------------------- 2. Column Rename + Structure ------------------
COLUMN_MAP = {f'OpSet{i}': f'op_setting_{i}' for i in range(1,4)}
COLUMN_MAP.update({f'Sensor{i}': f'sensor_measurement_{i}' for i in range(1,22)})
COLUMN_MAP['Label_RUL_30'] = 'RUL_binary'

def load_and_structure_data(file_path, fake_units):
    df = pd.read_csv(file_path)
    df.rename(columns=COLUMN_MAP, inplace=True)

    total_rows = len(df)

    # FIX → use ceil so fake_units * cycles >= total_rows
    cycles_per_unit = int(np.ceil(total_rows / fake_units))

    df['unit_number'] = np.repeat(range(1, fake_units+1), cycles_per_unit)[:total_rows]
    df['time_in_cycles'] = np.tile(range(1, cycles_per_unit+1), fake_units)[:total_rows]

    return df, cycles_per_unit

# IMPORTANT FIX: set fake_units so cycles >= 15
df_train, train_cycles = load_and_structure_data(train_path, fake_units=1000)
df_test, test_cycles   = load_and_structure_data(test_path,  fake_units=400)

print("\nTrain cycles/unit:", train_cycles)
print("Test cycles/unit :", test_cycles)

# -------------------- 3. Scaling -------------------------------
from sklearn.preprocessing import MinMaxScaler

selected_sensors = [2,3,4,7,11,12,15,20,21]
feature_cols = [f'op_setting_{i}' for i in range(1,4)] + [
    f'sensor_measurement_{i}' for i in selected_sensors
]

scaler = MinMaxScaler()
df_train[feature_cols] = scaler.fit_transform(df_train[feature_cols])
df_test[feature_cols] = scaler.transform(df_test[feature_cols])

# -------------------- 4. Class Weights -------------------------
import torch

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)

labels = df_train['RUL_binary']
class_counts = labels.value_counts().sort_index()
weights = torch.tensor([len(labels)/(2*c) for c in class_counts]).float().to(DEVICE)

print("Class distribution:", class_counts.to_dict())

# -------------------- 5. Dataset (15-cycle window) --------------
from torch.utils.data import Dataset, DataLoader

CONTEXT_LENGTH = 15

class CNNDataset(Dataset):
    def __init__(self, df):
        self.samples = []
        for unit in df['unit_number'].unique():
            u = df[df['unit_number']==unit].sort_values('time_in_cycles')
            for i in range(CONTEXT_LENGTH-1, len(u)):
                ctx = u.iloc[i-CONTEXT_LENGTH+1:i+1]
                x = ctx[feature_cols].values.astype(np.float32)  # (15,12)
                y = int(ctx['RUL_binary'].iloc[-1])
                self.samples.append((x, y))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        x, y = self.samples[idx]
        x = torch.tensor(x).permute(1, 0)   # → [12, 15]
        return x, torch.tensor(y)

train_dataset = CNNDataset(df_train)
test_dataset  = CNNDataset(df_test)

print("\nTrain samples:", len(train_dataset))
print("Test samples :", len(test_dataset))

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=32)

# -------------------- 6. CNN Model -----------------------------
import torch.nn as nn

class CNNBinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(12, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(64, 2)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.pool(x).squeeze(-1)
        return self.fc(x)

model = CNNBinaryClassifier().to(DEVICE)

# -------------------- 7. Train Loop -----------------------------
criterion = nn.CrossEntropyLoss(weight=weights)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

EPOCHS = 5
print("\n🚀 Training Started...\n")

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for x, y in train_loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS}  Loss: {total_loss/len(train_loader):.4f}")

print("\n🎉 Training Complete!")

# -------------------- 8. Evaluation ----------------------------
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def evaluate(model, loader):
    preds, trues = [], []
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            out = model(x)
            pred = torch.argmax(out, dim=1)
            preds.extend(pred.cpu().numpy())
            trues.extend(y.cpu().numpy())

    print("\n📌 Test Evaluation")
    print("Accuracy :", accuracy_score(trues, preds))
    print("Precision:", precision_score(trues, preds))
    print("Recall   :", recall_score(trues, preds))
    print("F1 Score :", f1_score(trues, preds))
    print("Confusion Matrix:\n", confusion_matrix(trues, preds))

evaluate(model, test_loader)

# -------------------- 9. Unit-wise Evaluation ------------------
def unitwise_predict(model, df):
    trues, preds = [], []
    model.eval()

    for unit in df['unit_number'].unique():
        u = df[df['unit_number']==unit].sort_values('time_in_cycles')
        ctx = u.tail(CONTEXT_LENGTH)
        x = torch.tensor(ctx[feature_cols].values.astype(np.float32)).permute(1,0).unsqueeze(0).to(DEVICE)

        with torch.no_grad():
            out = model(x)
            pred = torch.argmax(out, dim=1).item()

        trues.append(int(ctx['RUL_binary'].iloc[-1]))
        preds.append(pred)

    return np.array(trues), np.array(preds)

y_true, y_pred = unitwise_predict(model, df_test)

print("\n📌 Unit-wise Evaluation")
print("Accuracy :", accuracy_score(y_true, y_pred))
print("Precision:", precision_score(y_true, y_pred))
print("Recall   :", recall_score(y_true, y_pred))
print("F1 Score :", f1_score(y_true, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))

📌 Upload TRAIN dataset CSV


Saving synthetic_balanced_data_20000_60_40.csv to synthetic_balanced_data_20000_60_40 (3).csv

📌 Upload TEST dataset CSV


Saving synthetic_balanced_test_data_7000_50_50.csv to synthetic_balanced_test_data_7000_50_50 (3).csv

Train cycles/unit: 20
Test cycles/unit : 18
Using device: cpu
Class distribution: {0: 8000, 1: 12000}

Train samples: 6000
Test samples : 1554

🚀 Training Started...

Epoch 1/5  Loss: 0.6847
Epoch 2/5  Loss: 0.4768
Epoch 3/5  Loss: 0.1590
Epoch 4/5  Loss: 0.0767
Epoch 5/5  Loss: 0.0476

🎉 Training Complete!

📌 Test Evaluation
Accuracy : 0.9967824967824968
Precision: 0.9948652118100129
Recall   : 0.9987113402061856
F1 Score : 0.9967845659163987
Confusion Matrix:
 [[774   4]
 [  1 775]]

📌 Unit-wise Evaluation
Accuracy : 0.9974293059125964
Precision: 0.9948717948717949
Recall   : 1.0
F1 Score : 0.9974293059125964
Confusion Matrix:
 [[194   1]
 [  0 194]]
